In [2]:
import pandas as pd
import dbutil
import KlineService
import datetime
from BollBand import DayBollBand,WeekBollBand,MonthBollBand,SixtyMinBollBand,FiveMinBollBand
from IPython.core.debugger import set_trace
import talib

def clearBollBand(period):
    mydb = dbutil.connectDB()
    collection = mydb[chooseBollBandCollection(period)]
    collection.delete_many({})
    indexes = collection.index_information()
    if "code_1_date_1" in indexes.keys():
        collection.drop_index( "code_1_date_1" )
def createIndex(period):
    mydb = dbutil.connectDB()
    collection = mydb[chooseBollBandCollection(period)]
    collection.create_index( [("code", 1), ("date",1)])
##
#  选择不同的BollBand Collection
#
def chooseBollBandCollection(period):
    periodBollBandCollection = {
        "day" : "BollBand_Day",
        "week" : "BollBand_Week",
        "month" : "BollBand_Month",
        "5m" : "BollBand_5m",
        "15m" : "BollBand_15m",
        "30m" : "BollBand_30m",
        "60m" : "BollBand_60m"
    }
    return periodBollBandCollection.get(period)

##
#  写BollBand数据库
#
#
def writeBollBandToDb(period, stockCode, stockName, boll_df):
    dataList = []
    for index,bollBand in boll_df.iterrows():
        bollBandDate = bollBand['date']
        if period == "day":
            bollBandObj = DayBollBand(stockCode, stockName)
        elif period == "week":
            bollBandObj = WeekBollBand(stockCode, stockName)
        elif period == "month":
            bollBandObj = MonthBollBand(stockCode, stockName)
        elif period == "5m":
            bollBandObj = FiveMinBollBand(stockCode, stockName)
        elif period == "15m":
            bollBandObj = FiftyMinBollBand(stockCode, stockName)
        elif period == "30m":
            bollBandObj = ThirtyMinBollBand(stockCode, stockName)
        elif period == "60m":
            bollBandObj = SixtyMinBollBand(stockCode, stockName)

        bollBandObj.date = bollBandDate
        bollBandObj.upper = bollBand['upper']
        bollBandObj.middle = bollBand['middle']
        bollBandObj.lower = bollBand['lower']
        bollBandObj.closePrice = bollBand['closePrice']
        bollBandObj.overBuy = bollBand['overBuyFlag']
        bollBandObj.overSell = bollBand['overSellFlag']
        
        dataList.append(bollBandObj.__dict__)
        
    mydb = dbutil.connectDB()
    collection = mydb[chooseBollBandCollection(period)]
    if len(dataList) > 0:
        collection.insert_many(dataList)
    else:
        raise RuntimeError("BollBand数据为空")

##
#  计算个股各周期的布林线
#
#
def computeStockBollBand(period, stockCode, stockName, startDate, endDate):
    try:
#       compute1 = datetime.datetime.now().timestamp()
        df = KlineService.readStockKline(stockCode, period, startDate, endDate)
#       compute2 = datetime.datetime.now().timestamp()
#       print("read stockLine:", compute2 - compute1)
        if period == "day":
            # 剔除日线停盘数据
            df = df[df['tradeStatus'] == '1']
        closePrice = df["closePrice"]
        upper, middle, lower = talib.BBANDS(closePrice, timeperiod=22, nbdevup=2, nbdevdn=2, matype=0)
        data = {
            "upper":upper,
            "middle":middle,
            "lower":lower,
            "closePrice":closePrice
        }
        bollDF = pd.DataFrame(data)
        bollDF["date"] = df["date"]
        bollDF["overBuyFlag"] = False
        bollDF["overSellFlag"] = False
#       compute3 = datetime.datetime.now().timestamp()
#       print("compute rsi:", compute3 - compute2)
        writeBollBandToDb(period, stockCode, stockName, bollDF)
#       compute4 = datetime.datetime.now().timestamp()
#       print("write to db:", compute4 - compute3)
        return True
    except BaseException as e:
        print ("compute bollband " + stockCode + " error:" + str(e))
        return False
    
def computeAllStockBollBand(period, startDate):
    print("begin clear BollBand period:", period)
    clearBollBand(period)
    print("cleared BollBand period:", period)
    stockDict = KlineService.allStocks()
#     time2 = datetime.datetime.now().timestamp()
#     print("read stocks finished:",time2 - time1)
    endDate = str(datetime.date.today())
    jobStart = datetime.datetime.now().timestamp()
    processCount = 0
    failCount = 0
    jobTotal = len(stockDict)
    for key,stock in stockDict.items():
        processCount = processCount + 1
        #指数没有分钟线，调过指数的BollBand分钟线计算
        if period.endswith("m") and stock["stockType"] != 1:
            continue
        if not computeStockBollBand(period, key, stock["name"], startDate, endDate):
            failCount = failCount + 1
        if processCount % 100 == 0 and processCount > 0:
            print ("bollband process:", processCount, " of ", jobTotal ," failed:", failCount)
            
    jobFinished = datetime.datetime.now().timestamp()
    createIndex(period)
    print("write all stock BollBand to db finished, cost:", jobFinished - jobStart)

In [3]:
%%time
# computeStockBollBand("60m", "sz.002030", "达安基因", "2018-01-01", "2019-09-27")
computeAllStockBollBand("day", "2017-01-01")
computeAllStockBollBand("60m", "2018-01-01")

begin clear BollBand period: day
cleared BollBand period: day
bollband process: 100  of  4395  failed: 0
bollband process: 200  of  4395  failed: 0
compute bollband sh.600001 error:'tradeStatus'
compute bollband sh.600002 error:'tradeStatus'
compute bollband sh.600003 error:'tradeStatus'
compute bollband sh.600065 error:'tradeStatus'
bollband process: 300  of  4395  failed: 4
compute bollband sh.600087 error:'tradeStatus'
compute bollband sh.600092 error:'tradeStatus'
compute bollband sh.600102 error:'tradeStatus'
compute bollband sh.600145 error:inputs are all NaN
compute bollband sh.600181 error:'tradeStatus'
bollband process: 400  of  4395  failed: 9
compute bollband sh.600205 error:'tradeStatus'
compute bollband sh.600253 error:'tradeStatus'
compute bollband sh.600263 error:'tradeStatus'
compute bollband sh.600286 error:'tradeStatus'
compute bollband sh.600296 error:'tradeStatus'
bollband process: 500  of  4395  failed: 14
compute bollband sh.600357 error:'tradeStatus'
bollband pro

compute bollband sz.399953 error:'tradeStatus'
compute bollband sz.399954 error:'tradeStatus'
compute bollband sz.399956 error:'tradeStatus'
compute bollband sz.399960 error:'tradeStatus'
compute bollband sz.399962 error:'tradeStatus'
compute bollband sz.399968 error:'tradeStatus'
compute bollband sz.399980 error:'tradeStatus'
write all stock BollBand to db finished, cost: 861.7798550128937
CPU times: user 12min 16s, sys: 15.9 s, total: 12min 32s
Wall time: 16min 22s
